In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1958208146573307520, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8095788695790984428
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
import sys
import time
import math
import os
import pandas as pd
import numpy as np

import keras
from keras import layers, metrics, models, optimizers
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, concatenate, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 2
img_rows, img_cols = 256, 256
input_shape = (img_rows, img_cols, 3)
epochs = 150

In [6]:
train_dir = 'data_augmented/cvc300_train/'
validation_dir = 'data_augmented/cvc300_validation/'
test_dir = 'data_augmented/cvc300_test/'

image_folder = 'images'
masks_folder = 'masks'

In [7]:
from keras.layers import Input
from keras.layers.core import Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.models import Model


def Tiramisu(img_size):
    inp = Input(shape=(img_size, img_size, 3))

    x = Conv2D(48, kernel_size=(3, 3), padding='same',
               input_shape=(img_size, img_size, 3),
               kernel_initializer="he_uniform",
               kernel_regularizer=l2(0.0001))(inp)
    # (5 * 4)* 2 + 5 + 5 + 1 + 1 +1
    # growth_m = 4 * 12
    # previous_m = 48

    x = _denseBlock(x, 4, 96) # 4*12 = 48 + 48 = 96
    x = _transitionDown(x, 96)
    x = _denseBlock(x, 4, 144) # 4*12 = 48 + 96 = 144
    x = _transitionDown(x, 144)
    x = _denseBlock(x, 4, 192) # 4*12 = 48 + 144 = 192
    x = _transitionDown(x, 192)
    x = _denseBlock(x, 4, 240)# 4*12 = 48 + 192 = 240
    x = _transitionDown(x, 240)
    x = _denseBlock(x, 4, 288) # 4*12 = 48 + 288 = 336
    x = _transitionDown(x, 288)

    x = _denseBlock(x, 15, 336) # 4 * 12 = 48 + 288 = 336

    x = _transitionUp(x, 384)  # m = 288 + 4x12 + 4x12 = 384.
    x = _denseBlock(x, 4, 384)

    x = _transitionUp(x, 336) #m = 240 + 4x12 + 4x12 = 336
    x = _denseBlock(x, 4, 336)

    x = _transitionUp(x, 288) # m = 192 + 4x12 + 4x12 = 288
    x = _denseBlock(x, 4, 288)

    x = _transitionUp(x, 240) # m = 144 + 4x12 + 4x12 = 240
    x = _denseBlock(x, 4, 240)

    x = _transitionUp(x, 192) # m = 96 + 4x12 + 4x12 = 192
    x = _denseBlock(x, 4, 192)

    x = Conv2D(1, kernel_size=(1, 1), padding='same',
              kernel_initializer="he_uniform",
              kernel_regularizer=l2(0.0001))(x)
    x = Activation('sigmoid')(x)

    model = Model(inputs=inp, outputs=x)

    return model


def _denseBlock(x, layers, filters):
    for i in range(layers):
        x = BatchNormalization(gamma_regularizer=l2(0.0001),
                               beta_regularizer=l2(0.0001))(x)
        x = Activation('relu')(x)
        x = Conv2D(filters, (3, 3), padding='same',
                   kernel_initializer="he_uniform")(x)
        x = Dropout(0.2)(x)

    return x


def _transitionDown(x, filters):
    x = BatchNormalization(gamma_regularizer=l2(0.0001),
                           beta_regularizer=l2(0.0001))(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (1, 1), padding='same',
               kernel_initializer="he_uniform")(x)
    x = Dropout(0.2)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    return x


def _transitionUp(x, filters):
    x = Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding='same',
                        kernel_initializer="he_uniform")(x)

    return x

In [8]:
seed = 1142
data_gen_args = dict(rescale=1. / 255, featurewise_center=True, featurewise_std_normalization=True)
image_datagen = ImageDataGenerator(**data_gen_args)
masks_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    train_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    train_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_image_generator = image_datagen.flow_from_directory(
    validation_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_mask_generator = masks_datagen.flow_from_directory(
    validation_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_image_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_mask_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

Found 26112 images belonging to 1 classes.
Found 26112 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 28 images belonging to 1 classes.
Found 28 images belonging to 1 classes.


In [9]:
train_generator = zip(image_generator, mask_generator)
validation_generator = zip(validation_image_generator, validation_mask_generator)
test_generator = zip(test_image_generator, test_mask_generator)

In [10]:
nb_train_samples = len(image_generator.filenames)
nb_validation_samples = len(validation_image_generator.filenames)
nb_test_samples = len(test_image_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

nb_train_samples: 26112
nb_validation_samples: 8704
nb_test_samples: 28

predict_size_train: 13056
predict_size_validation: 4352
predict_size_test: 14


In [11]:
smooth = 1.
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def Specificity(y_true, y_pred):
    true_negatives = K.abs(y_pred)- K.abs(y_true)
    return ((true_negatives+smooth)/(y_pred+ smooth))

def Sensitivity(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return ((y_pred+smooth)/ (y_true+smooth))

def Jaccard_index(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall))

In [12]:
model = Tiramisu(img_size = 256)
model.summary()
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer= adam_opt, loss=dice_coef_loss, 
              metrics=[dice_coef, 'acc','mse', Jaccard_index, Specificity, Sensitivity, precision, recall, f1score])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 48)      1344      
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 48)      192       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 48)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 96)      41568     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256, 256, 96)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256, 256, 96)      384       
__________

In [13]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    validation_data = validation_generator,
                    validation_steps = nb_validation_samples//batch_size)

Epoch 1/150
13056/13056 [==============================] - 5523s 423ms/step - loss: 1.3301 - dice_coef: 0.1601 - acc: 0.7135 - mean_squared_error: 0.1586 - Jaccard_index: 0.0922 - Specificity: 0.9619 - Sensitivity: 1.1225 - precision: 0.1191 - recall: 0.3659 - f1score: nan - val_loss: 1.4365 - val_dice_coef: 2.1894e-04 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 2.1517e-04 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: nan
Epoch 2/150
13056/13056 [==============================] - 5493s 421ms/step - loss: 1.2070 - dice_coef: 0.1689 - acc: 0.7047 - mean_squared_error: 0.1694 - Jaccard_index: 0.0974 - Specificity: 0.9628 - Sensitivity: 1.1331 - precision: 0.1244 - recall: 0.4129 - f1score: nan - val_loss: 1.3125 - val_dice_coef: 2.1182e-04 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 2.1093e-04 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision:

Epoch 17/150
13056/13056 [==============================] - 5544s 425ms/step - loss: -0.0972 - dice_coef: 0.7889 - acc: 0.8336 - mean_squared_error: 0.0130 - Jaccard_index: 0.6600 - Specificity: 0.9730 - Sensitivity: 1.0049 - precision: 0.8656 - recall: 0.8528 - f1score: nan - val_loss: 0.5332 - val_dice_coef: 0.1479 - val_acc: 0.7776 - val_mean_squared_error: 0.0846 - val_Jaccard_index: 0.0874 - val_Specificity: 0.9478 - val_Sensitivity: 1.0241 - val_precision: 0.1925 - val_recall: 0.1701 - val_f1score: nan
Epoch 18/150
13056/13056 [==============================] - 5551s 425ms/step - loss: -0.1283 - dice_coef: 0.7986 - acc: 0.8341 - mean_squared_error: 0.0123 - Jaccard_index: 0.6727 - Specificity: 0.9732 - Sensitivity: 1.0047 - precision: 0.8756 - recall: 0.8626 - f1score: nan - val_loss: 0.4295 - val_dice_coef: 0.2299 - val_acc: 0.7655 - val_mean_squared_error: 0.0995 - val_Jaccard_index: 0.1421 - val_Specificity: 0.9521 - val_Sensitivity: 1.0479 - val_precision: 0.2402 - val_recall

KeyboardInterrupt: 

In [ ]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])

In [ ]:
prediction = model.predict_generator(test_generator, verbose=1, steps=nb_test_samples)

In [ ]:
prediction.shape

In [ ]:
test_results = model.evaluate_generator(test_generator, steps=50)

print(model.metrics_names)
print(test_results)

In [ ]:
coef = np.array(history.history['Jaccard_index'])
val_coef = np.array(history.history['val_Jaccard_index'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))

In [ ]:
import skimage.io as io

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

os.mkdir('05.U-Net2_results_dir')

In [ ]:
saveResult('05.U-Net2_results_dir',prediction)

In [ ]:
def normalizeData(img,mask):
    mean = np.mean(img)
    std = np.std(img)
    img -= mean
    img /= std
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img,mask)

test_data_dir= "data_augmented/cvc300_test/images/"
test_data_mask_dir = "data_augmented/cvc300_test/masks/"
test_data_pred_dir = "05.U-Net2_results_dir"
img_rows, img_cols = 256, 256

In [ ]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

file_names = next(os.walk(test_data_dir))[2]
scores = []
for file in file_names:
    grey_img = load_img(os.path.join(test_data_dir,file), target_size=(img_rows, img_cols), grayscale=False)
    mask_img = load_img(os.path.join(test_data_mask_dir,file), target_size=(img_rows, img_cols), grayscale=True)
    img = img_to_array(grey_img)
    img_mask = img_to_array(mask_img)
    
    img, img_mask = normalizeData(img, img_mask)
    img = np.reshape(img,(1,)+img.shape)
    
    pred = model.predict([img])
    sess = tf.Session()
    score = sess.run(Jaccard_index(img_mask, pred))
    print("{} -- jaccard index: {}".format(file,score))
    scores.append([file,score])

    result_img = array_to_img(pred[0] * 255 )
    result_img.save(os.path.join(test_data_pred_dir, file.split('.')[0] + '_predict.jpg'))

with open("unet_test_result.csv", 'w') as f:
    f.write("filename, jaccard_index\n")
    for i in range(len(scores)):
        #print(scores[i])
        f.write("{},{}\n".format(scores[i][0], scores[i][1]))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/102_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray")

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray", vmin = 0.0, vmax= 0.09)

In [ ]:
results_ =  np.squeeze(np.multiply(prediction[1], 255))
print(results_)

plt.imshow(results_, cmap = "gray", vmin = 0.0, vmax= 0.7)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/35_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice_coeff')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()